In [30]:
import pandas as pd
import dask.dataframe as dd

df = dd.read_csv("C:/Users/myaka/Desktop/Final_Year_project/data/http_processed.csv")

In [31]:
print(df.head())

                         id     user  \
0  {V1D3-W8BL16YA-2594OWGB}  ANC1950   
1  {C7A0-F6CK17IX-5508HGRY}  ANC1950   
2  {C0X3-I5RZ16GM-1535IAMY}  ANC1950   
3  {G8V6-M0CF17SO-9968THHW}  ANC1950   
4  {I9A3-Z4JU92SK-7362XXPN}  SAB1954   

                                                 url   activity  \
0  http://icio.us/John_Edward_Brownlee_as_Attorne...  WWW Visit   
1  http://babycenter.com/Manchester_SmallScale_Ex...  WWW Visit   
2  http://babycenter.com/Manchester_SmallScale_Ex...  WWW Visit   
3  http://timeanddate.com/Accurate_News_and_Infor...  WWW Visit   
4  http://timeanddate.com/Accurate_News_and_Infor...  WWW Visit   

                                             content  
0  Further consultation with post-production team...  
1  These two populations have been observed in ma...  
2  These two populations have been observed in ma...  
3  Cape sold the US rights to the recently formed...  
4  Cape sold the US rights to the recently formed...  


In [32]:
import dask.dataframe as dd
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
import pandas as pd

# Assuming `df` is a Dask DataFrame already loaded
# Drop the 'user' column
df = df.drop(columns=['user'])

# Remove rows where the 'activity' column is NaN
df = df.dropna(subset=['activity'])

# Extract numeric features from the 'content' and 'url' columns
df['content_length'] = df['content'].apply(len, meta=('x', 'f8'))
df['url_length'] = df['url'].apply(len, meta=('x', 'f8'))

# Compute final dataset before sorting
df = df.compute()

# Encode activity types
encoder = LabelEncoder()
df["activity_encoded"] = encoder.fit_transform(df["activity"])



# Combine all features into a single DataFrame
features = df[['content_length', 'url_length', 'activity_encoded']]

# Convert to pandas DataFrame for the next steps (you can perform Dask operations on smaller chunks)


# Initialize the anomaly detection model (Isolation Forest)
model = IsolationForest(contamination=0.1)  # You can adjust contamination based on the expected anomaly rate

# Train the model on the features
model.fit(features)

# Get anomaly scores (negative scores indicate outliers/anomalies)
anomaly_scores = model.decision_function(features)

# Normalize the anomaly scores to be between 0 and 10
scaler = MinMaxScaler(feature_range=(0, 10))
anomaly_scores_normalized = scaler.fit_transform(anomaly_scores.reshape(-1, 1))

# Add the anomaly scores to the original dataframe
df['anomaly_score'] = anomaly_scores_normalized.flatten()

# Output the final dataframe with anomaly scores
print(df[['id', 'anomaly_score']])


                             id  anomaly_score
0      {V1D3-W8BL16YA-2594OWGB}       5.919534
1      {C7A0-F6CK17IX-5508HGRY}       8.004178
2      {C0X3-I5RZ16GM-1535IAMY}       8.004178
3      {G8V6-M0CF17SO-9968THHW}       9.268736
4      {I9A3-Z4JU92SK-7362XXPN}       9.268736
...                         ...            ...
82845  {W2Q3-B7GC63MJ-6634UVHS}       8.349240
82846  {B7H9-X4MZ94CC-5550ZTRH}       9.039051
82847  {G4Q4-T2JT18OE-6607LLBJ}       8.679134
82848  {J8P0-I8LQ24HE-5414UGGB}       9.050909
82849  {O6X2-Z3EQ49YB-1172IOVU}       8.688117

[82850 rows x 2 columns]


In [33]:
import pickle

with open("anomaly_http.pkl", "wb") as model_file:
    pickle.dump(model, model_file)